- Chat GPT를 활용하여 생성한 데이터를 전처리하여 MySQL에 적재

In [2]:
import pandas as pd
import os

# 현재 노트북 파일이 있는 디렉토리 기준으로 dataset 폴더 내의 csv 파일 불러오기
notebook_dir = os.path.dirname(os.path.abspath("__file__"))  # 현재 작업 디렉토리 가져오기
csv_path = os.path.join(notebook_dir, "dataset", "2023_9월모평_분류적용.csv")  # 파일 경로 설정

# CSV 파일 로드
suneung_df = pd.read_csv(csv_path)

# 데이터 확인
print(suneung_df.head())

   수험번호 선택과목  문제번호  대분류      중분류          소분류  학생이 쓴 답  정답  배점
0     1   공통     1  수학Ⅰ   지수와 로그    로그 함수의 성질        4   4   2
1     1   공통     2  수학Ⅱ   함수의 극한   함수의 극한값 계산        1   1   2
2     1   공통     3  수학Ⅰ     삼각함수     삼각함수의 성질        2   2   2
3     1   공통     4  수학Ⅱ  함수의 연속성  연속의 개념 및 조건        1   1   3
4     1   공통     5  수학Ⅰ       수열    등차수열의 일반항        3   3   3


In [7]:
suneung_df.head(5)

,수험번호,선택과목,문제번호,대분류,중분류,소분류,학생이 쓴 답,정답,배점
0,1,공통,1,수학Ⅰ,지수와 로그,로그 함수의 성질,4,4,2
1,1,공통,2,수학Ⅱ,함수의 극한,함수의 극한값 계산,1,1,2
2,1,공통,3,수학Ⅰ,삼각함수,삼각함수의 성질,2,2,2
3,1,공통,4,수학Ⅱ,함수의 연속성,연속의 개념 및 조건,1,1,3
4,1,공통,5,수학Ⅰ,수열,등차수열의 일반항,3,3,3


In [ ]:
# 23번 이후 문항의 선택과목을 가져와서 1~22번 문항의 "공통"을 해당 선택과목으로 변경
mask = suneung_df["문제번호"] <= 22
selected_subject = suneung_df.loc[~mask, ["수험번호", "선택과목"]].drop_duplicates()

# 수험번호 기준으로 선택과목 매핑
suneung_df.loc[mask, "선택과목"] = suneung_df.loc[mask, "수험번호"].map(selected_subject.set_index("수험번호")["선택과목"])

# 결과 출력
print(suneung_df.head(30))

In [4]:
# 수험번호를 좀 더 현실감 있게 바꾸기 위해 20220000을 더함
suneung_df["수험번호"] = suneung_df["수험번호"] + 20220000

In [5]:
# 결과 확인
suneung_df.head(5)

,수험번호,선택과목,문제번호,대분류,중분류,소분류,학생이 쓴 답,정답,배점
0,20220001,미적분,1,수학Ⅰ,지수와 로그,로그 함수의 성질,4,4,2
1,20220001,미적분,2,수학Ⅱ,함수의 극한,함수의 극한값 계산,1,1,2
2,20220001,미적분,3,수학Ⅰ,삼각함수,삼각함수의 성질,2,2,2
3,20220001,미적분,4,수학Ⅱ,함수의 연속성,연속의 개념 및 조건,1,1,3
4,20220001,미적분,5,수학Ⅰ,수열,등차수열의 일반항,3,3,3


In [6]:
suneung_df.columns.tolist()

['수험번호', '선택과목', '문제번호', '대분류', '중분류', '소분류', '학생이 쓴 답', '정답', '배점']

In [3]:
import random

# 한국에서 흔한 성씨 리스트
surnames = ["김", "이", "박", "최", "정", "강", "조", "윤", "장", "임", "한", "오", "신", "서", "권", "황", "안", "송", "홍", "유"]  # 20개
# 흔한 이름 리스트 (중복 방지를 위해 다양한 조합 생성)
first_names = ["민수", "지훈", "서연", "현우", "지수", "영희", "철수", "하늘", "나래", "도현", "유진", "태희", "승현", "지안", "은우", "민지", "예준", "수빈", "도윤", "서현"]  # 20개

# 중복 방지를 위한 이름 조합 생성
def generate_unique_names(surnames, first_names, count):
    name_list = []
    while len(name_list) < count:
        name = random.choice(surnames) + random.choice(first_names)
        if name not in name_list:
            name_list.append(name)
    return sorted(name_list)  # 가나다순 정렬

# 수험번호 생성
exam_numbers = list(range(20220001, 20220301))  # 300개 생성
names = generate_unique_names(surnames, first_names, len(exam_numbers))

# 데이터프레임 생성
student_data = pd.DataFrame({"수험번호": exam_numbers, "이름": names})

In [4]:
# 결과 확인
student_data.head()

,수험번호,이름
0,20220001,강나래
1,20220002,강도윤
2,20220003,강도현
3,20220004,강민수
4,20220005,강민지


In [11]:
student_data.columns.tolist()

['수험번호', '이름']

In [2]:
import mysql.connector

# MySQL 연결 정보
host = "127.0.0.1"  # MySQL 서버 주소 (예: "your-server.com" 또는 "IP 주소")
user = "root"  # MySQL 사용자명
password = "9703"  # MySQL 비밀번호
database = "megastudy"  # 사용할 데이터베이스 이름

try:
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    cursor = conn.cursor()
    print("MySQL 연결 성공!")
except mysql.connector.Error as err:
    print(f"MySQL 연결 실패: {err}")

MySQL 연결 성공!


In [7]:
from sqlalchemy import create_engine

# MySQL 연결 정보
host = "127.0.0.1"
user = "root"
password = "9703"
database = "megastudy"

# SQLAlchemy를 사용하여 MySQL 연결 엔진 생성
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# 데이터프레임을 MySQL 테이블로 저장
table_name = "suneung_results"  # 저장할 테이블 이름

try:
    suneung_df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
    print(f"'{table_name}' 테이블에 데이터 저장 완료!")
except Exception as e:
    print(f"데이터 저장 실패: {e}")

'suneung_results' 테이블에 데이터 저장 완료!


In [12]:
# MySQL 연결 정보
host = "127.0.0.1"
user = "root"
password = "9703"
database = "megastudy"

# SQLAlchemy를 사용하여 MySQL 연결 엔진 생성
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# 데이터프레임을 MySQL 테이블로 저장
table_name = "student_info"  # 저장할 테이블 이름

try:
    student_data.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
    print(f"'{table_name}' 테이블에 데이터 저장 완료!")
except Exception as e:
    print(f"데이터 저장 실패: {e}")

'student_info' 테이블에 데이터 저장 완료!
